In [1]:
spark
base_df = spark.read.parquet('s3a://dse-cohort5-group5/wildfire_capstone/integratedData.pca.parquet.gz')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1586888643838_0003,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [33]:
!pip install matplotlib

invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    !pip install matplotlib
    ^
SyntaxError: invalid syntax



In [2]:
base_df.printSchema()

root
 |-- date: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pcaFeatures: vector (nullable = true)

In [3]:
base_df.show(3, vertical=True, truncate=True)

-RECORD 0---------------------------
 date        | 915235200000000000   
 latitude    | 32.608333333333334   
 longitude   | -116.93333330000002  
 pcaFeatures | [-18.953460978093... 
-RECORD 1---------------------------
 date        | 915235200000000000   
 latitude    | 32.775000000000006   
 longitude   | -117.26666663333334  
 pcaFeatures | [-27.538244860325... 
-RECORD 2---------------------------
 date        | 915235200000000000   
 latitude    | 32.775000000000006   
 longitude   | -116.14166663333334  
 pcaFeatures | [-3.5834800101858... 
only showing top 3 rows

In [5]:
base_df.createOrReplaceTempView("base_df")

In [37]:
import numpy as np
def abs_and_add_index(row):
    return list(enumerate(np.abs(row.pcaFeatures.toArray())))
pca_features = base_df.select("pcaFeatures").rdd.map(abs_and_add_index)

In [38]:
from pyspark.ml.feature import PCAModel
loadedModel = PCAModel.load("s3a://dse-cohort5-group5/wildfire_capstone/pcamodel.sparkobject")
loadedModel.explainedVariance

DenseVector([0.3134, 0.2013, 0.0769, 0.0522, 0.03, 0.024, 0.0235, 0.0217, 0.0207, 0.0158, 0.0124, 0.0111, 0.0097, 0.0084, 0.0081, 0.0077, 0.0069, 0.0067, 0.0062, 0.0052, 0.005, 0.005, 0.0042, 0.0041, 0.0037, 0.0034, 0.0033, 0.0031, 0.0029, 0.0026, 0.0026, 0.0025, 0.0023, 0.0023, 0.0021, 0.0021, 0.002, 0.0019, 0.0018, 0.0018, 0.0017, 0.0017, 0.0017, 0.0016, 0.0016, 0.0015, 0.0014, 0.0014, 0.0014, 0.0013, 0.0013, 0.0013, 0.0012, 0.0012, 0.0012, 0.0011, 0.0011, 0.0011, 0.0011, 0.001, 0.001, 0.001, 0.0009, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0007, 0.0007, 0.0007, 0.0007, 0.0007, 0.0007, 0.0007, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.00

In [39]:
list(enumerate(np.cumsum(loadedModel.explainedVariance.toArray()), 1))

[(1, 0.31342590735032066), (2, 0.5147006059156416), (3, 0.591555913707195), (4, 0.6437378658283267), (5, 0.6737564215045093), (6, 0.6977548239680709), (7, 0.7212389157608327), (8, 0.742938118500636), (9, 0.7636861310866209), (10, 0.7795163141016137), (11, 0.7919165771136883), (12, 0.8030094784439097), (13, 0.8127264664175387), (14, 0.8211551437652574), (15, 0.8293026473709858), (16, 0.8369569397346376), (17, 0.8438503361804398), (18, 0.850573280370734), (19, 0.8567453912940275), (20, 0.8619248340160028), (21, 0.8668952231484586), (22, 0.8718454860064142), (23, 0.8760940291981046), (24, 0.8802227067838937), (25, 0.8838760416470162), (26, 0.8872516862147937), (27, 0.8905567416535949), (28, 0.8936450885997287), (29, 0.8965828759232517), (30, 0.8992238651025747), (31, 0.9018183461781997), (32, 0.9042862988098589), (33, 0.9066277773730712), (34, 0.9088821305900862), (35, 0.9110083036384137), (36, 0.9130895212361592), (37, 0.9151101992082591), (38, 0.9169811626592441), (39, 0.918792998802618

In [40]:
pca_features.take(3)

[[(0, 18.95346097809354), (1, 0.06759753995620305), (2, 25.271759686743057), (3, 52.97100105677832), (4, 33.24635389906242), (5, 2.7913404255747083), (6, 27.579517110474217), (7, 21.725419325896194), (8, 7.198345477603096), (9, 8.073174025535295), (10, 27.17957990720802), (11, 12.544226708126098), (12, 6.56683632249648), (13, 19.105421209791807), (14, 11.077001883719626), (15, 23.98217266677097), (16, 3.5242673823531114), (17, 5.527945837770831), (18, 18.960039209879724), (19, 12.328236296564668), (20, 2.7113323310815605), (21, 8.738267679389502), (22, 10.292362752242415), (23, 0.7969439529059428), (24, 3.016321479489293), (25, 2.08941074029101), (26, 8.166114440629388), (27, 3.103287967883936), (28, 3.118184789699913), (29, 0.39306466307652743), (30, 4.300329334134786), (31, 0.3798633835896089), (32, 11.245135558185382), (33, 2.3025292523657455), (34, 2.1076122456726574), (35, 5.49894755093441), (36, 0.11064999352633646), (37, 4.257058802060664), (38, 3.1731364467481566), (39, 3.17415

In [41]:
def sort_and_take_top_N(vals):
    N = 31 # Represents 90% explained variance
    vals = sorted(vals, key=lambda x: -x[1])
    return vals[:N]
pca_features = pca_features.map(sort_and_take_top_N)

In [42]:
pca_features.take(3)

[[(3, 52.97100105677832), (4, 33.24635389906242), (6, 27.579517110474217), (10, 27.17957990720802), (2, 25.271759686743057), (15, 23.98217266677097), (7, 21.725419325896194), (13, 19.105421209791807), (18, 18.960039209879724), (0, 18.95346097809354), (11, 12.544226708126098), (19, 12.328236296564668), (32, 11.245135558185382), (14, 11.077001883719626), (52, 10.886995759402708), (22, 10.292362752242415), (47, 9.886898597262785), (46, 9.477625243529019), (57, 9.34686923173717), (58, 9.072986558538304), (44, 8.989090983694439), (21, 8.738267679389502), (95, 8.502249301011151), (54, 8.32311796041161), (26, 8.166114440629388), (9, 8.073174025535295), (67, 7.8910100248589155), (51, 7.802371761562069), (8, 7.198345477603096), (55, 6.831162962396601), (82, 6.7078675335387645)], [(3, 50.654069836046), (4, 31.24856132075419), (0, 27.53824486032517), (6, 27.346572366316735), (2, 24.46245929226524), (10, 24.302008099264143), (7, 22.914599682299453), (15, 21.102130102449625), (13, 18.09936956001927

In [53]:
top_indices = pca_features.flatMap(lambda xs: [(x[0], 1) for x in xs]).countByKey()
top_indices

defaultdict(<class 'int'>, {3: 5678702, 4: 5270186, 6: 5259679, 10: 4707350, 2: 6092483, 15: 3939700, 7: 5338360, 13: 4307523, 18: 3943957, 0: 6479570, 11: 4611243, 19: 3694220, 32: 2594189, 14: 4362429, 52: 1342487, 22: 3422724, 47: 1633535, 46: 1678666, 57: 1079632, 58: 1199212, 44: 1881775, 21: 3738764, 95: 255371, 54: 1299515, 26: 3076519, 9: 5041604, 67: 842008, 51: 1556444, 8: 5209326, 55: 1153849, 82: 511678, 71: 803002, 1: 6338181, 17: 4149083, 49: 1538148, 12: 4580302, 16: 4119640, 30: 2736161, 20: 3721987, 56: 1279969, 48: 1650592, 31: 2638598, 61: 1074174, 66: 926845, 106: 126779, 24: 3297888, 5: 5394337, 38: 2063892, 34: 2227784, 35: 2308660, 53: 1495866, 37: 1610961, 29: 2606054, 39: 674642, 28: 2854801, 69: 834948, 59: 1216133, 64: 923536, 45: 1691132, 63: 1017823, 60: 1176262, 23: 3456471, 27: 3017810, 43: 1376398, 76: 690260, 33: 2479673, 75: 719246, 36: 2262498, 139: 31696, 150: 16937, 129: 56293, 101: 227837, 148: 19316, 105: 170140, 107: 158592, 65: 948744, 25: 31207

In [54]:
top_indices_sorted = sorted(top_indices.items(), key=lambda x: -x[1])
top_indices_sorted

[(0, 6479570), (1, 6338181), (2, 6092483), (3, 5678702), (5, 5394337), (7, 5338360), (4, 5270186), (6, 5259679), (8, 5209326), (9, 5041604), (10, 4707350), (11, 4611243), (12, 4580302), (14, 4362429), (13, 4307523), (17, 4149083), (16, 4119640), (18, 3943957), (15, 3939700), (21, 3738764), (20, 3721987), (19, 3694220), (23, 3456471), (22, 3422724), (24, 3297888), (25, 3120744), (26, 3076519), (27, 3017810), (28, 2854801), (30, 2736161), (31, 2638598), (29, 2606054), (32, 2594189), (33, 2479673), (35, 2308660), (36, 2262498), (34, 2227784), (38, 2063892), (40, 2021227), (42, 1939551), (44, 1881775), (45, 1691132), (46, 1678666), (48, 1650592), (47, 1633535), (37, 1610961), (51, 1556444), (50, 1547793), (49, 1538148), (53, 1495866), (43, 1376398), (52, 1342487), (54, 1299515), (56, 1279969), (59, 1216133), (58, 1199212), (60, 1176262), (55, 1153849), (57, 1079632), (61, 1074174), (62, 1050530), (63, 1017823), (65, 948744), (66, 926845), (64, 923536), (68, 880793), (70, 870687), (67, 8420

In [52]:
# Col names from original PCA notebook, used to identify important columns
cols = ['fire_occurrences.`precipitation_amount_mm`', 
'fire_occurrences.`relative_humidity_%`', 
'fire_occurrences.`specific_humidity_kg/kg`', 
'fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north`', 
'fire_occurrences.`wind_speed_m/s`', 
'fire_occurrences.`max_air_temperature_K`', 
'fire_occurrences.`min_air_temperature_K`', 
'fire_occurrences.`burning_index_g_Unitless`', 
'fire_occurrences.`dead_fuel_moisture_100hr_Percent`', 
'fire_occurrences.`dead_fuel_moisture_1000hr_Percent`', 
'fire_occurrences.`energy_release_component-g_Unitless`', 
'fire_occurrences.`potential_evapotranspiration_mm`', 
'fire_occurrences.`mean_vapor_pressure_deficit_kPa`', 
'min.`7d_min_precipitation_amount_mm`', 
'min.`7d_min_relative_humidity_%`', 
'min.`7d_min_specific_humidity_kg/kg`', 
'min.`7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`7d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`7d_min_wind_speed_m/s`', 
'min.`7d_min_max_air_temperature_K`', 
'min.`7d_min_min_air_temperature_K`', 
'min.`7d_min_burning_index_g_Unitless`', 
'min.`7d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`7d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`7d_min_energy_release_component-g_Unitless`', 
'min.`7d_min_potential_evapotranspiration_mm`', 
'min.`7d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`14d_min_precipitation_amount_mm`', 
'min.`14d_min_relative_humidity_%`', 
'min.`14d_min_specific_humidity_kg/kg`', 
'min.`14d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`14d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`14d_min_wind_speed_m/s`', 
'min.`14d_min_max_air_temperature_K`', 
'min.`14d_min_min_air_temperature_K`', 
'min.`14d_min_burning_index_g_Unitless`', 
'min.`14d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`14d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`14d_min_energy_release_component-g_Unitless`', 
'min.`14d_min_potential_evapotranspiration_mm`', 
'min.`14d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`21d_min_precipitation_amount_mm`', 
'min.`21d_min_relative_humidity_%`', 
'min.`21d_min_specific_humidity_kg/kg`', 
'min.`21d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`21d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`21d_min_wind_speed_m/s`', 
'min.`21d_min_max_air_temperature_K`', 
'min.`21d_min_min_air_temperature_K`', 
'min.`21d_min_burning_index_g_Unitless`', 
'min.`21d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`21d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`21d_min_energy_release_component-g_Unitless`', 
'min.`21d_min_potential_evapotranspiration_mm`', 
'min.`21d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`30d_min_precipitation_amount_mm`', 
'min.`30d_min_relative_humidity_%`', 
'min.`30d_min_specific_humidity_kg/kg`', 
'min.`30d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`30d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`30d_min_wind_speed_m/s`', 
'min.`30d_min_max_air_temperature_K`', 
'min.`30d_min_min_air_temperature_K`', 
'min.`30d_min_burning_index_g_Unitless`', 
'min.`30d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`30d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`30d_min_energy_release_component-g_Unitless`', 
'min.`30d_min_potential_evapotranspiration_mm`', 
'min.`30d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`60d_min_precipitation_amount_mm`', 
'min.`60d_min_relative_humidity_%`', 
'min.`60d_min_specific_humidity_kg/kg`', 
'min.`60d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`60d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`60d_min_wind_speed_m/s`', 
'min.`60d_min_max_air_temperature_K`', 
'min.`60d_min_min_air_temperature_K`', 
'min.`60d_min_burning_index_g_Unitless`', 
'min.`60d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`60d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`60d_min_energy_release_component-g_Unitless`', 
'min.`60d_min_potential_evapotranspiration_mm`', 
'min.`60d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`90d_min_precipitation_amount_mm`', 
'min.`90d_min_relative_humidity_%`', 
'min.`90d_min_specific_humidity_kg/kg`', 
'min.`90d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`90d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`90d_min_wind_speed_m/s`', 
'min.`90d_min_max_air_temperature_K`', 
'min.`90d_min_min_air_temperature_K`', 
'min.`90d_min_burning_index_g_Unitless`', 
'min.`90d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`90d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`90d_min_energy_release_component-g_Unitless`', 
'min.`90d_min_potential_evapotranspiration_mm`', 
'min.`90d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`180d_min_precipitation_amount_mm`', 
'min.`180d_min_relative_humidity_%`', 
'min.`180d_min_specific_humidity_kg/kg`', 
'min.`180d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`180d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`180d_min_wind_speed_m/s`', 
'min.`180d_min_max_air_temperature_K`', 
'min.`180d_min_min_air_temperature_K`', 
'min.`180d_min_burning_index_g_Unitless`', 
'min.`180d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`180d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`180d_min_energy_release_component-g_Unitless`', 
'min.`180d_min_potential_evapotranspiration_mm`', 
'min.`180d_min_mean_vapor_pressure_deficit_kPa`', 
'min.`365d_min_precipitation_amount_mm`', 
'min.`365d_min_relative_humidity_%`', 
'min.`365d_min_specific_humidity_kg/kg`', 
'min.`365d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'min.`365d_min_wind_from_direction_Degrees_Clockwise_from_north`', 
'min.`365d_min_wind_speed_m/s`', 
'min.`365d_min_max_air_temperature_K`', 
'min.`365d_min_min_air_temperature_K`', 
'min.`365d_min_burning_index_g_Unitless`', 
'min.`365d_min_dead_fuel_moisture_100hr_Percent`', 
'min.`365d_min_dead_fuel_moisture_1000hr_Percent`', 
'min.`365d_min_energy_release_component-g_Unitless`', 
'min.`365d_min_potential_evapotranspiration_mm`', 
'min.`365d_min_mean_vapor_pressure_deficit_kPa`', 
'max.`7d_max_precipitation_amount_mm`', 
'max.`7d_max_relative_humidity_%`', 
'max.`7d_max_specific_humidity_kg/kg`', 
'max.`7d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`7d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`7d_max_wind_speed_m/s`', 
'max.`7d_max_max_air_temperature_K`', 
'max.`7d_max_min_air_temperature_K`', 
'max.`7d_max_burning_index_g_Unitless`', 
'max.`7d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`7d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`7d_max_energy_release_component-g_Unitless`', 
'max.`7d_max_potential_evapotranspiration_mm`', 
'max.`7d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`14d_max_precipitation_amount_mm`', 
'max.`14d_max_relative_humidity_%`', 
'max.`14d_max_specific_humidity_kg/kg`', 
'max.`14d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`14d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`14d_max_wind_speed_m/s`', 
'max.`14d_max_max_air_temperature_K`', 
'max.`14d_max_min_air_temperature_K`', 
'max.`14d_max_burning_index_g_Unitless`', 
'max.`14d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`14d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`14d_max_energy_release_component-g_Unitless`', 
'max.`14d_max_potential_evapotranspiration_mm`', 
'max.`14d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`21d_max_precipitation_amount_mm`', 
'max.`21d_max_relative_humidity_%`', 
'max.`21d_max_specific_humidity_kg/kg`', 
'max.`21d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`21d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`21d_max_wind_speed_m/s`', 
'max.`21d_max_max_air_temperature_K`', 
'max.`21d_max_min_air_temperature_K`', 
'max.`21d_max_burning_index_g_Unitless`', 
'max.`21d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`21d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`21d_max_energy_release_component-g_Unitless`', 
'max.`21d_max_potential_evapotranspiration_mm`', 
'max.`21d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`30d_max_precipitation_amount_mm`', 
'max.`30d_max_relative_humidity_%`', 
'max.`30d_max_specific_humidity_kg/kg`', 
'max.`30d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`30d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`30d_max_wind_speed_m/s`', 
'max.`30d_max_max_air_temperature_K`', 
'max.`30d_max_min_air_temperature_K`', 
'max.`30d_max_burning_index_g_Unitless`', 
'max.`30d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`30d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`30d_max_energy_release_component-g_Unitless`', 
'max.`30d_max_potential_evapotranspiration_mm`', 
'max.`30d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`60d_max_precipitation_amount_mm`', 
'max.`60d_max_relative_humidity_%`', 
'max.`60d_max_specific_humidity_kg/kg`', 
'max.`60d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`60d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`60d_max_wind_speed_m/s`', 
'max.`60d_max_max_air_temperature_K`', 
'max.`60d_max_min_air_temperature_K`', 
'max.`60d_max_burning_index_g_Unitless`', 
'max.`60d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`60d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`60d_max_energy_release_component-g_Unitless`', 
'max.`60d_max_potential_evapotranspiration_mm`', 
'max.`60d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`90d_max_precipitation_amount_mm`', 
'max.`90d_max_relative_humidity_%`', 
'max.`90d_max_specific_humidity_kg/kg`', 
'max.`90d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`90d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`90d_max_wind_speed_m/s`', 
'max.`90d_max_max_air_temperature_K`', 
'max.`90d_max_min_air_temperature_K`', 
'max.`90d_max_burning_index_g_Unitless`', 
'max.`90d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`90d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`90d_max_energy_release_component-g_Unitless`', 
'max.`90d_max_potential_evapotranspiration_mm`', 
'max.`90d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`180d_max_precipitation_amount_mm`', 
'max.`180d_max_relative_humidity_%`', 
'max.`180d_max_specific_humidity_kg/kg`', 
'max.`180d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`180d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`180d_max_wind_speed_m/s`', 
'max.`180d_max_max_air_temperature_K`', 
'max.`180d_max_min_air_temperature_K`', 
'max.`180d_max_burning_index_g_Unitless`', 
'max.`180d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`180d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`180d_max_energy_release_component-g_Unitless`', 
'max.`180d_max_potential_evapotranspiration_mm`', 
'max.`180d_max_mean_vapor_pressure_deficit_kPa`', 
'max.`365d_max_precipitation_amount_mm`', 
'max.`365d_max_relative_humidity_%`', 
'max.`365d_max_specific_humidity_kg/kg`', 
'max.`365d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'max.`365d_max_wind_from_direction_Degrees_Clockwise_from_north`', 
'max.`365d_max_wind_speed_m/s`', 
'max.`365d_max_max_air_temperature_K`', 
'max.`365d_max_min_air_temperature_K`', 
'max.`365d_max_burning_index_g_Unitless`', 
'max.`365d_max_dead_fuel_moisture_100hr_Percent`', 
'max.`365d_max_dead_fuel_moisture_1000hr_Percent`', 
'max.`365d_max_energy_release_component-g_Unitless`', 
'max.`365d_max_potential_evapotranspiration_mm`', 
'max.`365d_max_mean_vapor_pressure_deficit_kPa`', 
'mean.`7d_mean_precipitation_amount_mm`', 
'mean.`7d_mean_relative_humidity_%`', 
'mean.`7d_mean_specific_humidity_kg/kg`', 
'mean.`7d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`7d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`7d_mean_wind_speed_m/s`', 
'mean.`7d_mean_max_air_temperature_K`', 
'mean.`7d_mean_min_air_temperature_K`', 
'mean.`7d_mean_burning_index_g_Unitless`', 
'mean.`7d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`7d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`7d_mean_energy_release_component-g_Unitless`', 
'mean.`7d_mean_potential_evapotranspiration_mm`', 
'mean.`7d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`14d_mean_precipitation_amount_mm`', 
'mean.`14d_mean_relative_humidity_%`', 
'mean.`14d_mean_specific_humidity_kg/kg`', 
'mean.`14d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`14d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`14d_mean_wind_speed_m/s`', 
'mean.`14d_mean_max_air_temperature_K`', 
'mean.`14d_mean_min_air_temperature_K`', 
'mean.`14d_mean_burning_index_g_Unitless`', 
'mean.`14d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`14d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`14d_mean_energy_release_component-g_Unitless`', 
'mean.`14d_mean_potential_evapotranspiration_mm`', 
'mean.`14d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`21d_mean_precipitation_amount_mm`', 
'mean.`21d_mean_relative_humidity_%`', 
'mean.`21d_mean_specific_humidity_kg/kg`', 
'mean.`21d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`21d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`21d_mean_wind_speed_m/s`', 
'mean.`21d_mean_max_air_temperature_K`', 
'mean.`21d_mean_min_air_temperature_K`', 
'mean.`21d_mean_burning_index_g_Unitless`', 
'mean.`21d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`21d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`21d_mean_energy_release_component-g_Unitless`', 
'mean.`21d_mean_potential_evapotranspiration_mm`', 
'mean.`21d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`30d_mean_precipitation_amount_mm`', 
'mean.`30d_mean_relative_humidity_%`', 
'mean.`30d_mean_specific_humidity_kg/kg`', 
'mean.`30d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`30d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`30d_mean_wind_speed_m/s`', 
'mean.`30d_mean_max_air_temperature_K`', 
'mean.`30d_mean_min_air_temperature_K`', 
'mean.`30d_mean_burning_index_g_Unitless`', 
'mean.`30d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`30d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`30d_mean_energy_release_component-g_Unitless`', 
'mean.`30d_mean_potential_evapotranspiration_mm`', 
'mean.`30d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`60d_mean_precipitation_amount_mm`', 
'mean.`60d_mean_relative_humidity_%`', 
'mean.`60d_mean_specific_humidity_kg/kg`', 
'mean.`60d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`60d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`60d_mean_wind_speed_m/s`', 
'mean.`60d_mean_max_air_temperature_K`', 
'mean.`60d_mean_min_air_temperature_K`', 
'mean.`60d_mean_burning_index_g_Unitless`', 
'mean.`60d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`60d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`60d_mean_energy_release_component-g_Unitless`', 
'mean.`60d_mean_potential_evapotranspiration_mm`', 
'mean.`60d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`90d_mean_precipitation_amount_mm`', 
'mean.`90d_mean_relative_humidity_%`', 
'mean.`90d_mean_specific_humidity_kg/kg`', 
'mean.`90d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`90d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`90d_mean_wind_speed_m/s`', 
'mean.`90d_mean_max_air_temperature_K`', 
'mean.`90d_mean_min_air_temperature_K`', 
'mean.`90d_mean_burning_index_g_Unitless`', 
'mean.`90d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`90d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`90d_mean_energy_release_component-g_Unitless`', 
'mean.`90d_mean_potential_evapotranspiration_mm`', 
'mean.`90d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`180d_mean_precipitation_amount_mm`', 
'mean.`180d_mean_relative_humidity_%`', 
'mean.`180d_mean_specific_humidity_kg/kg`', 
'mean.`180d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`180d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`180d_mean_wind_speed_m/s`', 
'mean.`180d_mean_max_air_temperature_K`', 
'mean.`180d_mean_min_air_temperature_K`', 
'mean.`180d_mean_burning_index_g_Unitless`', 
'mean.`180d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`180d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`180d_mean_energy_release_component-g_Unitless`', 
'mean.`180d_mean_potential_evapotranspiration_mm`', 
'mean.`180d_mean_mean_vapor_pressure_deficit_kPa`', 
'mean.`365d_mean_precipitation_amount_mm`', 
'mean.`365d_mean_relative_humidity_%`', 
'mean.`365d_mean_specific_humidity_kg/kg`', 
'mean.`365d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'mean.`365d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 
'mean.`365d_mean_wind_speed_m/s`', 
'mean.`365d_mean_max_air_temperature_K`', 
'mean.`365d_mean_min_air_temperature_K`', 
'mean.`365d_mean_burning_index_g_Unitless`', 
'mean.`365d_mean_dead_fuel_moisture_100hr_Percent`', 
'mean.`365d_mean_dead_fuel_moisture_1000hr_Percent`', 
'mean.`365d_mean_energy_release_component-g_Unitless`', 
'mean.`365d_mean_potential_evapotranspiration_mm`', 
'mean.`365d_mean_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`7d_cumulative_precipitation_amount_mm`', 
'cumulative.`7d_cumulative_relative_humidity_%`', 
'cumulative.`7d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`7d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`7d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`7d_cumulative_wind_speed_m/s`', 
'cumulative.`7d_cumulative_max_air_temperature_K`', 
'cumulative.`7d_cumulative_min_air_temperature_K`', 
'cumulative.`7d_cumulative_burning_index_g_Unitless`', 
'cumulative.`7d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`7d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`7d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`7d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`7d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`14d_cumulative_precipitation_amount_mm`', 
'cumulative.`14d_cumulative_relative_humidity_%`', 
'cumulative.`14d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`14d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`14d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`14d_cumulative_wind_speed_m/s`', 
'cumulative.`14d_cumulative_max_air_temperature_K`', 
'cumulative.`14d_cumulative_min_air_temperature_K`', 
'cumulative.`14d_cumulative_burning_index_g_Unitless`', 
'cumulative.`14d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`14d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`14d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`14d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`14d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`21d_cumulative_precipitation_amount_mm`', 
'cumulative.`21d_cumulative_relative_humidity_%`', 
'cumulative.`21d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`21d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`21d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`21d_cumulative_wind_speed_m/s`', 
'cumulative.`21d_cumulative_max_air_temperature_K`', 
'cumulative.`21d_cumulative_min_air_temperature_K`', 
'cumulative.`21d_cumulative_burning_index_g_Unitless`', 
'cumulative.`21d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`21d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`21d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`21d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`21d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`30d_cumulative_precipitation_amount_mm`', 
'cumulative.`30d_cumulative_relative_humidity_%`', 
'cumulative.`30d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`30d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`30d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`30d_cumulative_wind_speed_m/s`', 
'cumulative.`30d_cumulative_max_air_temperature_K`', 
'cumulative.`30d_cumulative_min_air_temperature_K`', 
'cumulative.`30d_cumulative_burning_index_g_Unitless`', 
'cumulative.`30d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`30d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`30d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`30d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`30d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`60d_cumulative_precipitation_amount_mm`', 
'cumulative.`60d_cumulative_relative_humidity_%`', 
'cumulative.`60d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`60d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`60d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`60d_cumulative_wind_speed_m/s`', 
'cumulative.`60d_cumulative_max_air_temperature_K`', 
'cumulative.`60d_cumulative_min_air_temperature_K`', 
'cumulative.`60d_cumulative_burning_index_g_Unitless`', 
'cumulative.`60d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`60d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`60d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`60d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`60d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`90d_cumulative_precipitation_amount_mm`', 
'cumulative.`90d_cumulative_relative_humidity_%`', 
'cumulative.`90d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`90d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`90d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`90d_cumulative_wind_speed_m/s`', 
'cumulative.`90d_cumulative_max_air_temperature_K`', 
'cumulative.`90d_cumulative_min_air_temperature_K`', 
'cumulative.`90d_cumulative_burning_index_g_Unitless`', 
'cumulative.`90d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`90d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`90d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`90d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`90d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`180d_cumulative_precipitation_amount_mm`', 
'cumulative.`180d_cumulative_relative_humidity_%`', 
'cumulative.`180d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`180d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`180d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`180d_cumulative_wind_speed_m/s`', 
'cumulative.`180d_cumulative_max_air_temperature_K`', 
'cumulative.`180d_cumulative_min_air_temperature_K`', 
'cumulative.`180d_cumulative_burning_index_g_Unitless`', 
'cumulative.`180d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`180d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`180d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`180d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`180d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'cumulative.`365d_cumulative_precipitation_amount_mm`', 
'cumulative.`365d_cumulative_relative_humidity_%`', 
'cumulative.`365d_cumulative_specific_humidity_kg/kg`', 
'cumulative.`365d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'cumulative.`365d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 
'cumulative.`365d_cumulative_wind_speed_m/s`', 
'cumulative.`365d_cumulative_max_air_temperature_K`', 
'cumulative.`365d_cumulative_min_air_temperature_K`', 
'cumulative.`365d_cumulative_burning_index_g_Unitless`', 
'cumulative.`365d_cumulative_dead_fuel_moisture_100hr_Percent`', 
'cumulative.`365d_cumulative_dead_fuel_moisture_1000hr_Percent`', 
'cumulative.`365d_cumulative_energy_release_component-g_Unitless`', 
'cumulative.`365d_cumulative_potential_evapotranspiration_mm`', 
'cumulative.`365d_cumulative_mean_vapor_pressure_deficit_kPa`', 
'std.`7d_std_precipitation_amount_mm`', 
'std.`7d_std_relative_humidity_%`', 
'std.`7d_std_specific_humidity_kg/kg`', 
'std.`7d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`7d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`7d_std_wind_speed_m/s`', 
'std.`7d_std_max_air_temperature_K`', 
'std.`7d_std_min_air_temperature_K`', 
'std.`7d_std_burning_index_g_Unitless`', 
'std.`7d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`7d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`7d_std_energy_release_component-g_Unitless`', 
'std.`7d_std_potential_evapotranspiration_mm`', 
'std.`7d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`14d_std_precipitation_amount_mm`', 
'std.`14d_std_relative_humidity_%`', 
'std.`14d_std_specific_humidity_kg/kg`', 
'std.`14d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`14d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`14d_std_wind_speed_m/s`', 
'std.`14d_std_max_air_temperature_K`', 
'std.`14d_std_min_air_temperature_K`', 
'std.`14d_std_burning_index_g_Unitless`', 
'std.`14d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`14d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`14d_std_energy_release_component-g_Unitless`', 
'std.`14d_std_potential_evapotranspiration_mm`', 
'std.`14d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`21d_std_precipitation_amount_mm`', 
'std.`21d_std_relative_humidity_%`', 
'std.`21d_std_specific_humidity_kg/kg`', 
'std.`21d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`21d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`21d_std_wind_speed_m/s`', 
'std.`21d_std_max_air_temperature_K`', 
'std.`21d_std_min_air_temperature_K`', 
'std.`21d_std_burning_index_g_Unitless`', 
'std.`21d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`21d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`21d_std_energy_release_component-g_Unitless`', 
'std.`21d_std_potential_evapotranspiration_mm`', 
'std.`21d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`30d_std_precipitation_amount_mm`', 
'std.`30d_std_relative_humidity_%`', 
'std.`30d_std_specific_humidity_kg/kg`', 
'std.`30d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`30d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`30d_std_wind_speed_m/s`', 
'std.`30d_std_max_air_temperature_K`', 
'std.`30d_std_min_air_temperature_K`', 
'std.`30d_std_burning_index_g_Unitless`', 
'std.`30d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`30d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`30d_std_energy_release_component-g_Unitless`', 
'std.`30d_std_potential_evapotranspiration_mm`', 
'std.`30d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`60d_std_precipitation_amount_mm`', 
'std.`60d_std_relative_humidity_%`', 
'std.`60d_std_specific_humidity_kg/kg`', 
'std.`60d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`60d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`60d_std_wind_speed_m/s`', 
'std.`60d_std_max_air_temperature_K`', 
'std.`60d_std_min_air_temperature_K`', 
'std.`60d_std_burning_index_g_Unitless`', 
'std.`60d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`60d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`60d_std_energy_release_component-g_Unitless`', 
'std.`60d_std_potential_evapotranspiration_mm`', 
'std.`60d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`90d_std_precipitation_amount_mm`', 
'std.`90d_std_relative_humidity_%`', 
'std.`90d_std_specific_humidity_kg/kg`', 
'std.`90d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`90d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`90d_std_wind_speed_m/s`', 
'std.`90d_std_max_air_temperature_K`', 
'std.`90d_std_min_air_temperature_K`', 
'std.`90d_std_burning_index_g_Unitless`', 
'std.`90d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`90d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`90d_std_energy_release_component-g_Unitless`', 
'std.`90d_std_potential_evapotranspiration_mm`', 
'std.`90d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`180d_std_precipitation_amount_mm`', 
'std.`180d_std_relative_humidity_%`', 
'std.`180d_std_specific_humidity_kg/kg`', 
'std.`180d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`180d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`180d_std_wind_speed_m/s`', 
'std.`180d_std_max_air_temperature_K`', 
'std.`180d_std_min_air_temperature_K`', 
'std.`180d_std_burning_index_g_Unitless`', 
'std.`180d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`180d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`180d_std_energy_release_component-g_Unitless`', 
'std.`180d_std_potential_evapotranspiration_mm`', 
'std.`180d_std_mean_vapor_pressure_deficit_kPa`', 
'std.`365d_std_precipitation_amount_mm`', 
'std.`365d_std_relative_humidity_%`', 
'std.`365d_std_specific_humidity_kg/kg`', 
'std.`365d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 
'std.`365d_std_wind_from_direction_Degrees_Clockwise_from_north`', 
'std.`365d_std_wind_speed_m/s`', 
'std.`365d_std_max_air_temperature_K`', 
'std.`365d_std_min_air_temperature_K`', 
'std.`365d_std_burning_index_g_Unitless`', 
'std.`365d_std_dead_fuel_moisture_100hr_Percent`', 
'std.`365d_std_dead_fuel_moisture_1000hr_Percent`', 
'std.`365d_std_energy_release_component-g_Unitless`', 
'std.`365d_std_potential_evapotranspiration_mm`', 
'std.`365d_std_mean_vapor_pressure_deficit_kPa`']

In [55]:
N = 20
for idx, count in top_indices_sorted:
    print("Column {} was a top feature {} times".format(cols[idx], count))

Column fire_occurrences.`precipitation_amount_mm` was a top feature 6479570 times
Column fire_occurrences.`relative_humidity_%` was a top feature 6338181 times
Column fire_occurrences.`specific_humidity_kg/kg` was a top feature 6092483 times
Column fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2` was a top feature 5678702 times
Column fire_occurrences.`wind_speed_m/s` was a top feature 5394337 times
Column fire_occurrences.`min_air_temperature_K` was a top feature 5338360 times
Column fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north` was a top feature 5270186 times
Column fire_occurrences.`max_air_temperature_K` was a top feature 5259679 times
Column fire_occurrences.`burning_index_g_Unitless` was a top feature 5209326 times
Column fire_occurrences.`dead_fuel_moisture_100hr_Percent` was a top feature 5041604 times
Column fire_occurrences.`dead_fuel_moisture_1000hr_Percent` was a top feature 4707350 times
Column fire_occurrences.`energy_release_compon